# Hybrid Search 실습
이번 실습에서는 키워드 기반 검색(BM25)과 임베딩 기반 검색을 결합한 Hybrid Search를 구성하고 비교 실험을 진행합니다.

## 1. Hybrid Search란?
- **BM25**는 키워드 중심의 전통적인 정보 검색 알고리즘
- **Embedding**은 의미 기반 벡터 검색
- 두 방식을 결합하면 단어 일치와 의미 유사성 모두를 반영 가능

## 2. 텍스트 로딩 및 분할 (간단하게 진행)

In [ ]:
!pip install rank_bm25

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever

with open("kr_constitution_cleaned.txt", "r", encoding="utf-8") as f:
    text = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.create_documents([text])

## 3. 개별 검색기 구성 (BM25 + FAISS)

In [16]:
# BM25 (단어 기반)
bm25 = BM25Retriever.from_documents(docs)
bm25.k = 3

In [17]:
# FAISS (임베딩 기반)
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
faiss = FAISS.from_documents(docs, embedding)
faiss_retriever = faiss.as_retriever(search_kwargs={"k": 3})

## 4. Hybrid Retriever 구성

In [18]:
# 두 검색기를 가중치로 결합 (0~1 사이)
hybrid = EnsembleRetriever(retrievers=[bm25, faiss_retriever], weights=[0.5, 0.5])

## 5. 검색 결과 비교

In [ ]:
query = "헌법에서 대통령의 임기에 대해 알려줘"

print("\n[BM25 검색 결과]")
for i, d in enumerate(bm25.get_relevant_documents(query), 1):
    print(f"{i}.", d.page_content.strip()[:100])

print("\n[FAISS 검색 결과]")
for i, d in enumerate(faiss_retriever.get_relevant_documents(query), 1):
    print(f"{i}.", d.page_content.strip()[:100])

print("\n[Hybrid 검색 결과]")
for i, d in enumerate(hybrid.get_relevant_documents(query), 1):
    print(f"{i}.", d.page_content.strip()[:100])

## 6. 정리
- Hybrid Search는 키워드 기반 BM25와 의미 기반 Embedding 검색을 결합해 보완적인 검색 품질을 제공합니다.
- 실제 서비스에서는 가중치를 조절하거나 필터링 조건을 함께 적용해 더욱 정교한 검색이 가능합니다.